<a href="https://colab.research.google.com/github/Rakin96/my_code/blob/main/project_on_csv_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import seaborn as sns
import google.colab
from google.colab import files
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
uploaded=files.upload()


Saving train.csv to train.csv


In [ ]:
trd=pd.read_csv(io.BytesIO(uploaded['train.csv']))
trd1=trd.drop(columns=['PassengerId','Name','Cabin','Ticket'],axis=1)
again=files.upload()
test=pd.read_csv(io.BytesIO(again['test.csv']))

Saving test.csv to test.csv


In [ ]:
test1=test.drop(columns=['PassengerId','Name','Cabin','Ticket'],axis=1)
LE=preprocessing.LabelEncoder()
trd1.iloc[:,2]=LE.fit_transform(trd1['Sex'])
test1.iloc[:,1]=LE.fit_transform(test1['Sex'])
#test1.head()
test1['Fare'].fillna(test1['Fare'].mean(), inplace = True)
trd1['Age'].interpolate(method='linear',inplace=True)
test1['Age'].interpolate(method='linear',inplace=True)
trd1['Embarked'].fillna(trd1['Embarked'].mode(),inplace=True)
trd1['Embarked']=pd.get_dummies(trd1['Embarked'])
test1['Embarked']=pd.get_dummies(test1['Embarked'])
Xtr=trd1.iloc[:,1:]
Ytr=trd1.iloc[:,0]
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(Xtr,Ytr, random_state = 0)


In [ ]:
model=tf.keras.Sequential([tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(units=7,input_dim=7,activation='relu'),
                           tf.keras.layers.Dense(units=64,activation='relu'),
                           tf.keras.layers.Dense(units=128,activation='relu'),
                           tf.keras.layers.Dense(units=256,activation='relu'),
                           tf.keras.layers.Dense(512,activation='relu'),
                           tf.keras.layers.Dense(768,activation='relu'),
                           tf.keras.layers.Dense(12,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')
                           ])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history=model.fit(X_train,y_train,epochs=5,batch_size=20,validation_data=(X_val,y_val))

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

34/34 [==============================] - 0s 14ms/step - loss: 0.6676 - accuracy: 0.6138 - val_loss: 0.6030 - val_accuracy: 0.6233
Epoch 2/5
34/34 [==============================] - 0s 8ms/step - loss: 0.6263 - accuracy: 0.6138 - val_loss: 0.6257 - val_accuracy: 0.6233
Epoch 3/5
34/34 [==============================] - 0s 8ms/step - loss: 0.6587 - accuracy: 0.6153 - val_loss: 0.5972 - val_accuracy: 0.6233
Epoch 4/5
34/34 [==============================] - 0s 8ms/step - loss: 0.6246 - accuracy: 

In [ ]:
tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)


In [ ]:
#this portion is for data visualization
#plt.figure(figsize=(7,7))
#sns.lineplot(data=df1)#////for line plot
#sns.barplot(x=df1.index,y=df1['Age']) # for barplot(like in school days :v)
#sns.heatmap(df1.corr(),annot=True)
#sns.distplot(a=df1['Age'],kde=False)#here input arguement is a=input_data, and this is used for density distribution plot,this is also a frequency distribution 
#sns.kdeplot(data=df1['Age'],shade=True)# this show the frequency distribution of data, and here input arguement is data=....

In [ ]:
predictions=model.predict(test1)
predictions=predictions.round()
type(predictions)
y=test['PassengerId'].to_numpy()
y.resize(418,1)
predictions=predictions.astype(int)

In [ ]:
output_dt = pd.DataFrame({'PassengerId':y[:,0],'Survived': predictions[:,0]})
output_dt.to_csv('submission3.csv',index=False)
